In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
#from icecube import dataio, dataclasses
import os
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D,\
 BatchNormalization, MaxPooling2D,Convolution3D,MaxPooling3D, Merge, Input, Merge
from keras import regularizers
from configparser import ConfigParser
import resource
import tables

Using TensorFlow backend.


In [36]:
file_handler = dataio.I3File('/data/ana/PointSource/PS/IC86_2012/files/sim/2012/neutrino-generator/11029/00000-00999/Final_v2_nugen_numu_IC86.2012.011029.000000.i3.bz2')

In [6]:
frame_handler = file_handler.pop_physics()

In [7]:
I3Tree = frame_handler['I3MCTree']
MostEnergeticTrack = frame_handler['MCMostEnergeticTrack']

In [16]:
print frame_handler['I3MCWeightDict']['OneWeight']

12533920294.4


In [142]:
truncated_energy = 0
for i in I3Tree:
    interaction_type = str(i.type)
    if interaction_type in ['DeltaE','PairProd','Brems','EMinus']:
        truncated_energy += i.energy

In [143]:
print truncated_energy
print MostEnergeticTrack.energy

835.002519335
14050.1181641


In [118]:
print I3Tree

41198 NuMu (159146m ,-6.30758e+06m ,-7.27702e+06m) (139.074deg ,271.448deg) -3.21153e+07ns 557.518GeV 9.63073e+06m 
  41199 MuMinus (-242.713m ,-631.156m ,-475.833m) (138.529deg ,271.754deg) 9322.57ns 422.342GeV 1093.75m 
    41202 PairProd (-243.023m ,-621.021m ,-464.361m) (138.529deg ,271.754deg) 9373.64ns 1.94884GeV 0m 
    41203 DeltaE (-243.342m ,-610.613m ,-452.58m) (138.529deg ,271.754deg) 9426.09ns 0.576262GeV 0m 
    41204 PairProd (-243.514m ,-604.997m ,-446.222m) (138.529deg ,271.754deg) 9454.39ns 27.467GeV 0m 
    41205 DeltaE (-243.871m ,-593.323m ,-433.008m) (138.529deg ,271.754deg) 9513.22ns 0.799563GeV 0m 
    41206 PairProd (-244.131m ,-584.833m ,-423.398m) (138.529deg ,271.754deg) 9556ns 3.02639GeV 0m 
    41207 DeltaE (-245.182m ,-550.506m ,-384.541m) (138.529deg ,271.754deg) 9728.98ns 0.821513GeV 0m 
    41208 DeltaE (-245.635m ,-535.711m ,-367.794m) (138.529deg ,271.754deg) 9803.54ns 1.06655GeV 0m 
    41209 DeltaE (-245.657m ,-535.002m ,-366.992m) (138.529deg ,271

# Test Model

In [4]:
def add_layer(model, layer, args, kwargs):
    eval('model.add({}(*args,**kwargs))'.format(layer))
    return

def base_model(conf_model_file):
  inp_shape = []
  with open(conf_model_file) as f:
      args = []
      kwargs = dict()
      layer = ''
      mode = 'args'
      models = dict()
      cur_model = None
      cur_model_name = ''
      for line in f:
          cur_line = line.strip()
          if cur_model_name =='' and not cur_line == '':
            cur_model = Sequential()
            if cur_line[0]=='{' and cur_line[-1]=='}':
                cur_model_name = cur_line[1:-1]
            else:
                cur_model_name = 'model'                    
          if cur_line=='' and layer!='':
              if not layer == 'Merge':
                  add_layer(cur_model, layer, args,kwargs)
                  if 'input_shape' in kwargs.keys():
                      inp_shape.append(kwargs['input_shape'])
              else:
                  merge_layer_names = [name.strip() for name in kwargs['layers'][1:-1].split(',')]
                  kwargs = dict()
                  kwargs['mode']='concat'
                  add_layer(cur_model, layer,[[models[name] for name in merge_layer_names]], kwargs)
                  for name in merge_layer_names:
                    del models[name] 
              args = []
              kwargs = dict()
              mode = 'args'
              layer = ''
          elif cur_line == '' and layer == '':
            continue
          elif cur_line[0]=='{' and cur_line[-1]=='}':
              print(cur_model.summary())
              models[cur_model_name] = cur_model
              cur_model = Sequential()
              cur_model_name = cur_line[1:-1]
          elif cur_line[0] == '#':
              continue
          elif cur_line == '[kwargs]':
              mode = 'kwargs'
          elif layer == '':
              layer = cur_line[1:-1]
          elif mode == 'args':
              try:
                  args.append(eval(cur_line.split('=')[1].strip()))
              except:
                  args.append(cur_line.split('=')[1].strip())
          elif mode == 'kwargs':
              split_line = cur_line.split('=')
              try:
                  kwargs[split_line[0].strip()] = eval(split_line[1].strip())
              except:
                  kwargs[split_line[0].strip()] = split_line[1].strip()
      if layer != '':
          add_layer(cur_model, layer, args,kwargs)
      models[cur_model_name] = cur_model
  print(cur_model.summary())      
  return models[models.keys()[0]], eval(str(inp_shape))

In [2]:
conf_model_file = os.path.join('.', 'Networks', 'test.cfg')

In [6]:
model , inp_shape = base_model(conf_model_file)

NameError: name 'Settings' is not defined

In [135]:
print inp_shape

[(20, 10, 60, 1), (1,)]


In [101]:
adam = keras.optimizers.Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=adam, metrics=['accuracy'])

In [102]:
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_23 (Merge)             (None, 4001)              0         
_________________________________________________________________
dense_93 (Dense)             (None, 1)                 4002      
Total params: 63,237
Trainable params: 63,045
Non-trainable params: 192
_________________________________________________________________
None


In [16]:
branch1 = Sequential()
branch1.add(Dense(1, input_shape =  (1,), init = 'normal', activation = 'relu'))

branch2 = Sequential()
branch2.add(Dense(4, input_shape =  (4,), init = 'normal', activation = 'relu'))
branch2.add(BatchNormalization())
branch2.add(Dense(4, init = 'normal', activation = 'relu'))
branch2.add(BatchNormalization())
branch2.add(Dense(4, init = 'normal', activation = 'relu'))
branch2.add(BatchNormalization())
            
model = Sequential()
model.add(Merge([branch1, branch2], mode = 'concat'))

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="relu", kernel_initializer="normal", input_shape=(1,))`
  
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="relu", kernel_initializer="normal", input_shape=(4,))`
  """
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="relu", kernel_initializer="normal")`
  import sys
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="relu", kernel_initializer="normal")`
  if __name__ == '__main__':
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`

In [17]:
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_3 (Merge)              (None, 5)                 0         
Total params: 110
Trainable params: 86
Non-trainable params: 24
_________________________________________________________________
None


In [12]:
models = dict()

In [15]:
print len(models)

1


In [14]:
models['test'] = 2

In [3]:
f = open(conf_model_file)
config_array = f.read().splitlines()
config_blocks = []
single_block = []
for line in config_array:
    if line=='':
        config_blocks.append(single_block)
        single_block = []
    else:
        single_block.append(line)
settings =[]
model = []
mode = ''
for block in config_blocks:
    if mode =='' or block[0][0]=='*':
        if block[0] == '*Settings*':
            mode = 'settings'
        elif block[0] == '*Model*':
            mode = 'model'
        else:
            raise Exception('config file is corrupted')
    else:
        if mode=='settings':
            settings.append(block)
        elif mode=='model':
            model.append(block)

In [4]:
model_settings = settings

In [5]:
print model_settings

[['{Inputs}'], ['[Branch1]', 'variables = charge,time', 'transformations = (x-np.mean(x))/np.std(x), x'], ['[Branch2]', 'variables = charge', 'transformations = np.sum(x)'], ['{Outputs}', 'variables = np.log10(energy)']]


In [6]:
inp_variables = []
transformations = []
shapes = []
shape_names = []
mode == ''
if len(model_settings) == 0:
    model_settings = [['{Inputs}'], ['[model]', 'variables = [charge]', 'transformations = [x]']]
for block in model_settings:
    if block[0][0]=='{' and block[0][-1]=='}':
        mode=block[0][1:-1]
        continue
    if mode == 'Inputs':
        for element in block:
            if element[0]=='[' and element[-1]==']':
                shape_names.append(element[1:-1])
            else:
                split_row = element.split('=')
                if split_row[0].strip() == 'variables':
                    inp_variables.append(split_row[1].strip().split(','))
                elif split_row[0].strip() == 'transformations':
                    transformations.append(split_row[1].strip().split(','))
    else:
        continue
        
# for i in range(len(shape_names)):
#     temp_shape_arr = []
#     for j in range(len(inp_variables[i])):
#         temp_shape_arr.append('test')
#         if len(np.unique(temp_shape_arr))==1:
#             shapes.append(temp_shape_arr[0:-1]+(len(temp_shape_arr),))
#         else:
#             raise Expection('The transformations that you applied do not results in the same shape!!!!')
            
        


                
                
                

In [7]:
print shape_names
print inp_variables
print transformations


['Branch1', 'Branch2']
[['charge', 'time'], ['charge']]
[['(x-np.mean(x))/np.std(x)', ' x'], ['np.sum(x)']]


In [63]:
def base_model(model_def):
    models = dict()
    cur_model = None
    cur_model_name = ''
    for block in model_def:
        if block[0][0] == '{' and block[0][-1] == '}' or cur_model == None:
            if cur_model != None:
                print(cur_model.summary())
                models[cur_model_name] = cur_model    
            cur_model = Sequential()
            if block[0][0] == '{' and block[0][-1] == '}':
                cur_model_name = block[0][1:-1]
            else:
                cur_model_name = 'model'   
        if block[0][0] == '[' and block[0][-1] == ']':
            args = []
            kwargs = dict()
            layer = ''
            mode = 'args'
            layer=block[0][1:-1]
            for i in range(1,len(block)):
                if block[i]=='[kwargs]':
                    mode = 'kwargs'
                elif mode == 'args':
                    try:
                        args.append(eval(block[i].split('=')[1].strip()))
                    except:
                        args.append(block[i].split('=')[1].strip())
                elif mode == 'kwargs':
                    split_line = block[i].split('=')
                    try:
                        kwargs[split_line[0].strip()] = eval(split_line[1].strip())
                    except:
                        kwargs[split_line[0].strip()] = split_line[1].strip()   
            if not layer == 'Merge':
                add_layer(cur_model, layer, args,kwargs)
            else:
                merge_layer_names = [name.strip() for name in kwargs['layers'][1:-1].split(',')]
                kwargs = dict()
                kwargs['mode']='concat'
                add_layer(cur_model, layer,[[models[name] for name in merge_layer_names]], kwargs)
                for name in merge_layer_names:
                    del models[name] 
    print(cur_model.summary())
    models[cur_model_name] = cur_model  
    return cur_model

In [62]:
base_model(model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 10, 10, 10, 64)    3136      
_________________________________________________________________
batch_normalization_5 (Batch (None, 10, 10, 10, 64)    256       
_________________________________________________________________
activation_5 (Activation)    (None, 10, 10, 10, 64)    0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 10, 10, 64)    0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 10, 10, 10, 32)    55328     
_________________________________________________________________
batch_normalization_6 (Batch (None, 10, 10, 10, 32)    128       
_________________________________________________________________
activation_6 (Activation)    (None, 10, 10, 10, 32)    0         
__________

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.


In [73]:
test = (1,2,3)

In [69]:
(0,)+test

(0, 1, 2, 3)

In [84]:
test[0:-1]+(2,)

(1, 2, 2)

In [42]:
import h5py

In [54]:
hfile = h5py.File('./training_data/11069_06000-06999.h5')
pyh5file = tables.open_file('./training_data/11069_06000-06999.h5', 'r')

In [3]:
parser = ConfigParser()

In [4]:
parser.read('config.cfg')

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  """Entry point for launching an IPython kernel.


['config.cfg']

In [64]:
test = hfile['reco_vals'][0:10]

In [65]:
print test['energy']

[ 1255446.95250087    62737.8540757     11590.39518676    12298.58372246
    37500.921875     113917.40066847   449789.1281364    185191.82143345
   174527.04305926  1563133.19498762]


In [36]:
a = parser['azimuth']

In [20]:
print a['out_type']

ClosedNodeError: the node object is closed

In [38]:
import tables

In [ ]:
os.makedirs('./bla1/bla2/bla3/')

In [67]:
dtype = []
data_source = []
for i, key in enumerate(parser.keys()):
    if key == 'DEFAULT' or key =='Basics':
        continue
    cut = [-np.inf, np.inf]
    print parser[key].keys()
    if 'min' in parser[key].keys():
        cut[0] = eval(parser[key]['min'])
    if 'max' in parser[key].keys():
        cut[1] = eval(parser[key]['max'])  
    if 'variable' in parser[key].keys():
        data_source.append(('variable', parser[key]['variable'], cut))
    elif 'function' in parser[key].keys():
        data_source.append(('function', parser[key]['function']))        
    dtype.append((str(key), eval('np.'+parser[key]['out_type'])))
dtype=np.dtype(dtype)

[u'variable', u'out_type']
[u'variable', u'out_type']
[u'variable', u'out_type']
[u'variable', u'out_type']
[u'function', u'out_type']
[u'variable', u'out_type', u'min', u'max']


In [68]:
print data_source

[('variable', u"['MCMostEnergeticTrack'].dir.azimuth", [-inf, inf]), ('variable', u"['MCMostEnergeticTrack'].dir.zenith", [-inf, inf]), ('variable', u"['SplineMPEMuEXDifferential'].energy", [-inf, inf]), ('variable', u"['I3MCWeightDict']['OneWeight']", [-inf, inf]), ('function', u'calc_depositedE(x)'), ('variable', u"['MCMostEnergeticTrack'].energy", [100, 1000000000.0])]


In [18]:
if os.path.exists('test.h5'):
    os.remove('test.h5')
h5file = tables.open_file('test.h5', mode = "w", title = "Events for training the NN")

In [19]:
h5file.root._v_attrs.shape = (1,1,1)
h5file.root._v_attrs.len = 10000

In [20]:
reco_vals = tables.Table(h5file.root, 'reco_vals', description = dtype)
reco_vals.append(np.array([1.,2.,3.,.4,5.,6.]))
reco_vals.append(np.array([5.,6.,7.,8.,9.,16.]))
reco_vals.append(np.random.normal(-1,1, size=6))

In [21]:
h5file.close()

In [2]:
h5file = tables.open_file('test.h5', mode = "r", title = "Events for training the NN")

In [52]:
type(h5file.root._v_attrs.shape)

tuple

In [3]:
a= h5file.root.reco_vals

In [4]:
a.cols.energy[2]

-0.73662813583819886

In [10]:
h5file.close()

In [73]:
print len(dtype)

6


In [68]:
dtype2 = np.dtype([('azimuth', eval('np.float64')), ('zenith', np.float64)])

In [11]:
print len(hfile['charge'])
print len(pyh5file.root.charge)

207254
207254


In [11]:
inds = np.arange(0,len(hfile['charge']),len(hfile['charge'])/100)

In [12]:
import psutil
process = psutil.Process(os.getpid())
print('{}'.format(process.memory_info().rss/1e6))

211.103744


In [13]:
for i in range(4):
    t =[]
    print('{}'.format(process.memory_info().rss/1e9))
    print('{}'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1e9))
    t.extend(hfile['charge'][inds[i]:inds[i+1]])
    print('{}'.format(process.memory_info().rss/1e9))
    print('{}'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1e9))
    print ' --------- '

0.211103744
0.211103744
0.423591936
0.423591936
 --------- 
0.224677888
0.423591936
0.429596672
0.429596672
 --------- 
0.230682624
0.429596672
0.430354432
0.430354432
 --------- 
0.231440384
0.430354432
0.432054272
0.432054272
 --------- 


In [14]:
for i in range(4):
    t =[]
    print('{}'.format(process.memory_info().rss/1e9))
    print('{}'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1e9))
    t.extend(pyh5file.root.charge[inds[i]:inds[i+1]])
    print('{}'.format(process.memory_info().rss/1e9))
    print('{}'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1e9))
    print ' --------- '

0.233140224
0.432054272
0.43790336
0.43790336
 --------- 
0.238989312
0.43790336
0.439222272
0.439222272
 --------- 
0.240308224
0.439222272
0.441823232
0.441823232
 --------- 
0.242909184
0.441823232
0.441884672
0.441884672
 --------- 


In [57]:
test = [1,]

In [59]:
print test[]

IndexError: list index out of range

In [70]:
a= []
a.append(1)

In [2]:
test =' x , y ,abs' 

In [6]:
ary = test.split(',')
print ary
print [i.strip() for i in ary]

[' x ', ' y ', 'abs']
['x', 'y', 'abs']


In [1]:
from keras.layers import *

In [7]:
from configparser import ConfigParser

In [8]:
cfg_pars = ConfigParser()

In [9]:
cfg_pars.read('./../configs/default_new.cfg')

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  """Entry point for launching an IPython kernel.


['./../configs/default_new.cfg']

In [10]:
cfg_pars.keys()

[u'DEFAULT',
 u'Basics',
 u'Input_Charges',
 u'Input_Times',
 u'Input_Waveforms1',
 u'Cuts',
 u'depositedE',
 u'hitDOMs',
 u'firstParticle',
 u'energyFirstParticle',
 u'zenith',
 u'RunID',
 u'EventID',
 u'TauDecayLength',
 u'ClassificationLabel',
 u'StartingLabel',
 u'UpDownLabel',
 u'CoincidenceLabel',
 u'MinBias_Filter',
 u'Cascade_Filter',
 u'Muon_Filter']

In [11]:
x = cfg_pars['Input_Charges']
y =cfg_pars['Input_Times']
z = cfg_pars['Input_Waveforms1']

In [31]:
def replace_with_var(x):
    if ('(' in x) and (')' in x): 
        y = x[x.index('('):x.index(')')]
        x = x.replace(y, y.replace('c', 'charges').replace('t', 'times').replace('w', 'width'))
    else:
        x=x.replace('c', 'charges').replace('t', 'times').replace('w', 'width')
    return x

In [32]:
inputs = []
for key in x.keys():
    inputs.append((key, replace_with_var(x[key])))
for key in y.keys():
    inputs.append((key, replace_with_var(y[key])))
for q in z['quantiles'].split(','):
    inputs.append(('{}_{}_pct_charge_quantile'.format(z['type'], q.strip().replace('.','_')),
                   'fu.wf_quantiles(waveform, {})[\'{}\']'.format(q, z['type'])))

np.sum(c)
c[0]
len(c)
np.amin(t)
np.amax(t)-np.amin(t)
average(c, 1./w)
skew(t)
moment(times, moment=2)


In [33]:
print inputs

[(u'charge', u'np.sum(charges)'), (u'first_charge', u'charges[0]'), (u'num_pulses', u'len(charges)'), (u'time_first', u'np.amin(times)'), (u'time_spread', u'np.amax(times)-np.amin(times)'), (u'av_time_charges', u'average(charges, 1./width)'), (u'time_kurtosis', u'skew(times)'), (u'time_moment_2', u'moment(timesimes, momentimes=2)'), ('ATWD_0_05_pct_charge_quantile', "fu.wf_quantiles(waveform, 0.05)['ATWD']"), ('ATWD_0_1_pct_charge_quantile', "fu.wf_quantiles(waveform,  0.1)['ATWD']"), ('ATWD_0_15_pct_charge_quantile', "fu.wf_quantiles(waveform,  0.15)['ATWD']"), ('ATWD_0_2_pct_charge_quantile', "fu.wf_quantiles(waveform,  0.2)['ATWD']"), ('ATWD_0_25_pct_charge_quantile', "fu.wf_quantiles(waveform,  0.25)['ATWD']"), ('ATWD_0_3_pct_charge_quantile', "fu.wf_quantiles(waveform,  0.3)['ATWD']"), ('ATWD_0_35_pct_charge_quantile', "fu.wf_quantiles(waveform,  0.35)['ATWD']"), ('ATWD_0_4_pct_charge_quantile', "fu.wf_quantiles(waveform,  0.4)['ATWD']"), ('ATWD_0_45_pct_charge_quantile', "fu.wf_q